This is main project notebook

In [1]:
!conda install -c conda-forge wikipedia --yes
#important libraries
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import wikipedia as wp
import folium
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


print ('Hello Capstone Project Course!')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Hello Capstone Project Course!


In [2]:
#Get the html source and clean
html = wp.page("Demographics of Toronto neighbourhoods").html().encode("UTF-8")
df = pd.read_html(html)[1]
df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))
df.drop(columns = 'Map', inplace = True)
df = df.reindex(df.index.drop(1))
df = df.reset_index()
df.drop(columns = ['index'], inplace = True)
df.head()


,Name,FM,Census Tracts,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Second most common language (after English) by name,Second most common language (after English) by percentage
0,Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...",44577,12.45,3580,4.6,25750,11.1,5.9,Cantonese (19.3%),19.3% Cantonese
1,Alderwood,E,"0211.00, 0212.00",11656,4.94,2360,-4.0,35239,8.8,8.5,Polish (6.2%),06.2% Polish
2,Alexandra Park,OCoT,0039.00,4355,0.32,13609,0.0,19687,13.8,28.0,Cantonese (17.9%),17.9% Cantonese
3,Allenby,OCoT,0140.00,2513,0.58,4333,-1.0,245592,5.2,3.4,Russian (1.4%),01.4% Russian
4,Amesbury,NY,"0280.00, 0281.01, 0281.02",17318,3.51,4934,1.1,27546,16.4,19.7,Spanish (6.1%),06.1% Spanish


In [24]:
#getting latitiude and longitude
df['lat'] = 0
df['lon']= 0
df['lat'] = df['lat'].astype(np.float64)
df['lon'] = df['lon'].astype(np.float64)


for i in df.index:

    address = df.loc[i, 'Name'] + ' ,Toronto, Canada'

    geolocator = Nominatim()
    location = geolocator.geocode(address)
    if location != None:
        
        latitude = location.latitude
        longitude = location.longitude
        df['lat'][i]=latitude
        df['lon'][i]=longitude

print ('done')

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

done


In [5]:
#deleting unnecessary rows
a = df['lat'] == 0 
for i in df.index:
    if a[i] == True:
        df = df.reindex(df.index.drop(i))

df = df.reset_index(drop = True)


        

In [17]:
df.shape

(158, 14)

In [8]:
# create map of Toronto using latitude and longitude values
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, land_area in zip(df['lat'], df['lon'], df['Name'], df['Land area (km2)']):
    label = '{}, {} km2'.format(neighborhood, land_area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [1]:
#confidential
CLIENT_ID = '*****' # your Foursquare ID
CLIENT_SECRET = '*****' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500



In [10]:
#function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=df['Name'],
                                   latitudes=df['lat'],
                                   longitudes=df['lon']
                                  )


Agincourt
Alderwood
Alexandra Park
Allenby
Amesbury
Armour Heights
Banbury
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods â Steeles
Bedford Park
Bendale
Birch Cliff
Bloor West Village
Bracondale Hill
Branson
Bridle Path
Brockton
Cabbagetown
Caribou Park
Carleton Village
Casa Loma
Centennial
Chaplin Estates
Christie Pits
Church and Wellesley
Clairlea
Cliffcrest
Cliffside
Corktown
Crescent Town
Cricket Club
Davenport
Davisville
Deer Park
Discovery District
Don Mills
Don Valley Village
Dorset Park
Dovercourt Park
Downsview
Dufferin Grove
Earlscourt
East Danforth
Eatonville
Eglinton East
Elia (Jane and Finch)
Eringate
Fairbank
Fashion District
Financial District
Flemingdon Park
Forest Hill
Fort York/Liberty Village
Garden District
Glen Park
Grange Park
Graydon Hall
Guildwood
Harbord Village
Harbourfront / CityPlace
Harwood
Henry Farm
High Park North
Highland Creek
Hillcrest
Hoggs Hollow
Humber Heights
Humber Summit
Humber Valley Village
Humberlea
Humberwood
Ionview
Kens

In [12]:
print(toronto_venues.shape)
toronto_venues.head()

(3636, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.785353,-79.278549,Maple Yip Seafood 陸羽海鮮酒家,43.784752,-79.277787,Chinese Restaurant
1,Agincourt,43.785353,-79.278549,One2 Snacks,43.787048,-79.276658,Asian Restaurant
2,Agincourt,43.785353,-79.278549,Tim Hortons,43.785637,-79.279215,Coffee Shop
3,Agincourt,43.785353,-79.278549,In Cheon House Korean & Japanese Restaurant 인천관,43.786468,-79.275693,Korean Restaurant
4,Agincourt,43.785353,-79.278549,Beef Noodle Restaurant 老李牛肉麵,43.785937,-79.276031,Chinese Restaurant


In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,11,11,11,11,11,11
Alderwood,11,11,11,11,11,11
Alexandra Park,100,100,100,100,100,100
Allenby,10,10,10,10,10,10
Amesbury,4,4,4,4,4,4
Armour Heights,3,3,3,3,3,3
Banbury,4,4,4,4,4,4
Bathurst Manor,5,5,5,5,5,5
Bay Street Corridor,54,54,54,54,54,54


In [35]:
df.to_csv("toronto.csv")

In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 308 uniques categories.


In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
1,Alderwood,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
2,Alexandra Park,0.0,0.000000,0.000000,0.000000,0.010000,0.0,0.0,0.000000,0.01,...,0.000000,0.010000,0.000000,0.0,0.01000,0.0,0.000000,0.00,0.010000,0.0
3,Allenby,0.0,0.000000,0.000000,0.100000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
4,Amesbury,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
5,Armour Heights,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
6,Banbury,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
7,Bathurst Manor,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
8,Bay Street Corridor,0.0,0.018519,0.018519,0.000000,0.018519,0.0,0.0,0.000000,0.00,...,0.018519,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0
9,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.00,0.000000,0.0


In [20]:
#printing top 5 venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.27
1      Asian Restaurant  0.18
2           Coffee Shop  0.09
3  Cantonese Restaurant  0.09
4         Train Station  0.09


----Alderwood----
            venue  freq
0     Pizza Place  0.18
1            Pool  0.09
2            Bank  0.09
3    Skating Rink  0.09
4  Sandwich Place  0.09


----Alexandra Park----
                           venue  freq
0                            Bar  0.10
1             Italian Restaurant  0.05
2                           Café  0.05
3         Furniture / Home Store  0.04
4  Vegetarian / Vegan Restaurant  0.04


----Allenby----
                  venue  freq
0         Big Box Store   0.1
1  Fast Food Restaurant   0.1
2    African Restaurant   0.1
3          Intersection   0.1
4        Discount Store   0.1


----Amesbury----
                venue  freq
0                Park  0.25
1         Coffee Shop  0.25
2  Athletics & Sports  0.25
3                Bank  0.25
4              Office 

In [21]:
#function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Asian Restaurant,Korean Restaurant,Coffee Shop,Cantonese Restaurant,Train Station,Electronics Store,Hong Kong Restaurant,Fish Market,Fish & Chips Shop
1,Alderwood,Pizza Place,Pool,Coffee Shop,Dance Studio,Bank,Sandwich Place,Pharmacy,Pub,Skating Rink,Gym
2,Alexandra Park,Bar,Italian Restaurant,Café,Furniture / Home Store,Vegetarian / Vegan Restaurant,Nightclub,Arepa Restaurant,Boutique,Bakery,Restaurant
3,Allenby,Restaurant,African Restaurant,Bookstore,Food,Discount Store,Fast Food Restaurant,Intersection,Café,Big Box Store,Fish & Chips Shop
4,Amesbury,Park,Athletics & Sports,Coffee Shop,Bank,Zoo,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
5,Armour Heights,Pharmacy,Deli / Bodega,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
6,Banbury,Park,Tennis Court,Auto Garage,Zoo,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
7,Bathurst Manor,Bagel Shop,Playground,Baseball Field,Convenience Store,Park,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
8,Bay Street Corridor,Sushi Restaurant,Japanese Restaurant,Café,Mediterranean Restaurant,Burger Joint,Bubble Tea Shop,Gastropub,Salon / Barbershop,Electronics Store,Hookah Bar
9,Bayview Village,Breakfast Spot,Fish Market,Bank,Outdoor Supply Store,Coffee Shop,Gas Station,Fast Food Restaurant,Hardware Store,Burger Joint,Sporting Goods Shop


In [23]:
#CLUSTERING
# set number of clusters
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 1, 2, 1, 0, 2, 2], dtype=int32)

In [25]:
#toronto_merged = pd.dataframe()

# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.head() # check the last columns!


In [27]:
neighborhoods_venues_sorted['lat'] = 0
neighborhoods_venues_sorted['lon']= 0
neighborhoods_venues_sorted['lat'] = neighborhoods_venues_sorted['lat'].astype(np.float64)
neighborhoods_venues_sorted['lon'] = neighborhoods_venues_sorted['lon'].astype(np.float64)


for i in neighborhoods_venues_sorted.index:

    address = neighborhoods_venues_sorted.loc[i, 'Neighborhood'] + ' ,Toronto, Canada'

    geolocator = Nominatim()
    location = geolocator.geocode(address)
    if location != None:
        
        latitude = location.latitude
        longitude = location.longitude
        neighborhoods_venues_sorted['lat'][i]=latitude
        neighborhoods_venues_sorted['lon'][i]=longitude

print ('done')

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

done


In [28]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,lat,lon
0,Agincourt,Chinese Restaurant,Asian Restaurant,Korean Restaurant,Coffee Shop,Cantonese Restaurant,Train Station,Electronics Store,Hong Kong Restaurant,Fish Market,Fish & Chips Shop,2,43.785353,-79.278549
1,Alderwood,Pizza Place,Pool,Coffee Shop,Dance Studio,Bank,Sandwich Place,Pharmacy,Pub,Skating Rink,Gym,2,43.601717,-79.545232
2,Alexandra Park,Bar,Italian Restaurant,Café,Furniture / Home Store,Vegetarian / Vegan Restaurant,Nightclub,Arepa Restaurant,Boutique,Bakery,Restaurant,2,43.650758,-79.404298
3,Allenby,Restaurant,African Restaurant,Bookstore,Food,Discount Store,Fast Food Restaurant,Intersection,Café,Big Box Store,Fish & Chips Shop,2,43.711351,-79.553424
4,Amesbury,Park,Athletics & Sports,Coffee Shop,Bank,Zoo,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,1,43.706162,-79.483368


In [30]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['lat'], neighborhoods_venues_sorted['lon'], neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
#examine clusters
#cluster 1
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,lat,lon
7,Bagel Shop,Park,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,0,43.763893,-79.456367
21,Pet Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,0,43.670721,-79.453278
27,Fast Food Restaurant,Ice Cream Shop,Grocery Store,Pharmacy,Diner,Intersection,Park,0,43.708823,-79.295986
31,Golf Course,Bus Line,Zoo,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,0,43.695403,-79.293099
45,Theater,Hotel,American Restaurant,Burrito Place,Convenience Store,Grocery Store,Gym,0,43.646284,-79.560001
54,Park,Coffee Shop,Café,Flea Market,Gym,Track,Trail,0,43.634877,-79.409813
63,Tennis Court,Farm,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,0,43.769509,-79.354296
64,Baseball Field,Other Great Outdoors,Park,Scenic Lookout,Café,Skating Rink,Pool,0,43.646281,-79.463821
69,Gym,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,0,43.760078,-79.571760
70,Bus Stop,Pharmacy,Bakery,Flea Market,Fish Market,Flower Shop,Fish & Chips Shop,0,43.666472,-79.524314


In [32]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,lat,lon
4,Park,Zoo,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,1,43.706162,-79.483368
6,Park,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,1,43.742796,-79.369957
13,Park,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,1,43.702112,-79.260091
15,Park,Bar,Bakery,Flea Market,Fish Market,Fish & Chips Shop,Flower Shop,1,43.676125,-79.428000
23,Bar,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,1,43.787491,-79.150768
53,Playground,Bank,Zoo,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,1,43.693559,-79.413902
73,Deli / Bodega,Filipino Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,1,43.735990,-79.276515
80,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,1,43.722079,-79.437507
116,Fast Food Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,1,43.804930,-79.165837
147,Park,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,1,43.693640,-79.521043


In [33]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,lat,lon
0,Chinese Restaurant,Cantonese Restaurant,Train Station,Electronics Store,Hong Kong Restaurant,Fish Market,Fish & Chips Shop,2,43.785353,-79.278549
1,Pizza Place,Bank,Sandwich Place,Pharmacy,Pub,Skating Rink,Gym,2,43.601717,-79.545232
2,Bar,Vegetarian / Vegan Restaurant,Nightclub,Arepa Restaurant,Boutique,Bakery,Restaurant,2,43.650758,-79.404298
3,Restaurant,Discount Store,Fast Food Restaurant,Intersection,Café,Big Box Store,Fish & Chips Shop,2,43.711351,-79.553424
5,Pharmacy,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,2,43.743944,-79.430851
8,Sushi Restaurant,Burger Joint,Bubble Tea Shop,Gastropub,Salon / Barbershop,Electronics Store,Hookah Bar,2,43.665275,-79.387528
9,Breakfast Spot,Coffee Shop,Gas Station,Fast Food Restaurant,Hardware Store,Burger Joint,Sporting Goods Shop,2,43.769197,-79.376662
10,Bank,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,2,43.788035,-79.380318
11,Rental Car Location,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,2,43.737388,-79.410925
12,Chinese Restaurant,Fast Food Restaurant,Dog Run,Grocery Store,Fish Market,Flea Market,Fish & Chips Shop,2,43.753520,-79.255336


In [34]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 3, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,lat,lon
48,Park,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,3,43.662273,-79.576516
75,Park,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,3,43.699539,-79.556346
110,Park,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,3,43.775504,-79.134976
146,Park,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,3,43.783362,-79.455836
